# Used Car Price Prediction

## 1) Problem statement.

* This dataset comprises used cars sold on cardehko.com in India as well as important features of these cars.
* If user can predict the price of the car based on input features.
* Prediction results can be used to give new seller the price suggestion based on market condition.

## 2) Data Collection.
* The Dataset is collected from scrapping from cardheko webiste
* The data consists of 13 column and 15411 rows.

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [3]:
df = pd.read_csv(r"./data/cardekho_imputated.csv", index_col=[0])
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


## Data Cleaning
### Handling Missing values

* Handling Missing values 
* Handling Duplicates
* Check data type
* Understand the dataset

In [4]:
# Checking for missing values
df.isna().sum()

car_name             0
brand                0
model                0
vehicle_age          0
km_driven            0
seller_type          0
fuel_type            0
transmission_type    0
mileage              0
engine               0
max_power            0
seats                0
selling_price        0
dtype: int64

In [5]:
# Dropping the car_name and brand columns
df = df.drop(columns=['car_name', 'brand'])
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [7]:
# Check the type of features
numerical_features = [feature for feature in df.columns if df[feature].dtype != 'O']
categorical_features = [feature for feature in df.columns if df[feature].dtype == 'O']
discrete_features = [feature for feature in numerical_features if len(df[feature].unique()) <= 25]
continuous_features = [feature for feature in numerical_features if feature not in discrete_features]

# Print the type of features
print(f'Total Number of Features: {len(df.columns)}')
print(f'Numerical Features: {len(numerical_features)}')
print(f'Categorical Features: {len(categorical_features)}')
print('\nTypes of Numerical Features')
print(f'Discrete Features: {len(discrete_features)}')
print(f'Continuous Features: {len(continuous_features)}')

Total Number of Features: 11
Numerical Features: 7
Categorical Features: 4

Types of Numerical Features
Discrete Features: 2
Continuous Features: 5


In [9]:
from sklearn.model_selection import train_test_split

# Drop the selling_price column and insert it at the end of the dataframe to use iloc
selling_price = df.pop('selling_price')
df['selling_price'] = selling_price

# Split data into dependent and independent variables
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [10]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [12]:
pd.DataFrame(y.head())

,selling_price
0,120000
1,550000
2,215000
3,226000
4,570000


## Feature Encoding and Scaling
**One Hot Encoding for Columns which had lesser unique values and not ordinal**
* One hot encoding is a process by which categorical variables are converted into a form that could be provided to ML algorithms to do a better job in prediction.

In [17]:
# Get all the count unique values from the categorical features
for feature in categorical_features:
    print(f'{feature}: {df[feature].nunique()}')

model: 120
seller_type: 3
fuel_type: 5
transmission_type: 2


In [13]:
# Check all categorical variables distribution
for column in df.select_dtypes(include='object').columns:
    print(df[column].value_counts())
    print()

model
i20            906
Swift Dzire    890
Swift          781
Alto           778
City           757
              ... 
Ghibli           1
Altroz           1
GTC4Lusso        1
Aura             1
Gurkha           1
Name: count, Length: 120, dtype: int64

seller_type
Dealer              9539
Individual          5699
Trustmark Dealer     173
Name: count, dtype: int64

fuel_type
Petrol      7643
Diesel      7419
CNG          301
LPG           44
Electric       4
Name: count, dtype: int64

transmission_type
Manual       12225
Automatic     3186
Name: count, dtype: int64



In [14]:
from sklearn.preprocessing import LabelEncoder

# Using the LabelEncoder to encode model, because of many unique values
# Create an object of the LabelEncoder
label_encoder = LabelEncoder()

# Encode the categorical features
X['model'] = label_encoder.fit_transform(X['model'])

In [15]:
X.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15411 entries, 0 to 19543
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   model              15411 non-null  object 
 1   vehicle_age        15411 non-null  int64  
 2   km_driven          15411 non-null  int64  
 3   seller_type        15411 non-null  object 
 4   fuel_type          15411 non-null  object 
 5   transmission_type  15411 non-null  object 
 6   mileage            15411 non-null  float64
 7   engine             15411 non-null  int64  
 8   max_power          15411 non-null  float64
 9   seats              15411 non-null  int64  
 10  selling_price      15411 non-null  int64  
dtypes: float64(2), int64(5), object(4)
memory usage: 1.4+ MB


In [18]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

# Create an object of the StandardScaler and OneHotEncoder
scaler = StandardScaler()
encoder = OneHotEncoder(drop='first')

# Create a ColumnTransformer
preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", encoder, cat_features),
        ("StandardScaler", scaler, num_features)
        
    ],remainder='passthrough' 
)

preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('OneHotEncoder', OneHotEncoder(drop='first'),
                                 Index(['seller_type', 'fuel_type', 'transmission_type'], dtype='object')),
                                ('StandardScaler', StandardScaler(),
                                 Index(['model', 'vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power',
       'seats'],
      dtype='object'))])

In [19]:
# Fit and transform the data
X = preprocessor.fit_transform(X)

In [20]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.519714,0.983562,1.247335,-0.000276,-1.324259,-1.263352,-0.403022
1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.225693,-0.343933,-0.690016,-0.192071,-0.554718,-0.432571,-0.403022
2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.536377,1.647309,0.084924,-0.647583,-0.554718,-0.479113,-0.403022
3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.519714,0.983562,-0.360667,0.292211,-0.936610,-0.779312,-0.403022
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.666211,-0.012060,-0.496281,0.735736,0.022918,-0.046502,-0.403022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15406,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.508844,0.983562,-0.869744,0.026096,-0.767733,-0.757204,-0.403022
15407,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.556082,-1.339555,-0.728763,-0.527711,-0.216964,-0.220803,2.073444
15408,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.407551,-0.012060,0.220539,0.344954,0.022918,0.068225,-0.403022
15409,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.426247,-0.343933,72.541850,-0.887326,1.329794,0.917158,2.073444


In [21]:
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Get the shape of the data
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((12328, 14), (3083, 14), (12328,), (3083,))

## Model Training And Model Selection

In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [23]:
# Create a function to evaluate the model
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    
    return mae, mse, rmse, r2_square

In [24]:
# Create a dictionary of models
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "Adaboost Regressor":AdaBoostRegressor()
}

# Loop through the models
for i in range(len(list(models))):
    # Get the model
    model = list(models.values())[i]
    # Fit the model
    model.fit(X_train, y_train)

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae, model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae, model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    # Print the results
    print(list(models.keys())[i])
    
    print('Model Performance for Training set')
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- Mean Squared Error: {:.4f}".format(model_train_mse))
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- R2 Score: {:.4f}".format(model_train_r2))
    print("- Adjusted R2 Score: {:.4f}".format(1 - (1-model_train_r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)))

    print('-'*55)
    
    print('Model Performance for Test set')
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- Mean Squared Error: {:.4f}".format(model_test_mse))
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    print("- Adjusted R2 Score: {:.4f}".format(1 - (1-model_test_r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)))
    
    print('='*55)
    print('\n')

Linear Regression
Model Performance for Training set
- Mean Absolute Error: 268101.6071
- Mean Squared Error: 306756099359.7596
- Root Mean Squared Error: 553855.6665
- R2 Score: 0.6218
- Adjusted R2 Score: 0.6213
-------------------------------------------------------
Model Performance for Test set
- Mean Absolute Error: 279618.5794
- Mean Squared Error: 252550062888.5656
- Root Mean Squared Error: 502543.5930
- R2 Score: 0.6645
- Adjusted R2 Score: 0.6630


Lasso
Model Performance for Training set
- Mean Absolute Error: 268099.2226
- Mean Squared Error: 306756104248.3742
- Root Mean Squared Error: 553855.6710
- R2 Score: 0.6218
- Adjusted R2 Score: 0.6213
-------------------------------------------------------
Model Performance for Test set
- Mean Absolute Error: 279614.7461
- Mean Squared Error: 252549134806.7813
- Root Mean Squared Error: 502542.6696
- R2 Score: 0.6645
- Adjusted R2 Score: 0.6630


Ridge
Model Performance for Training set
- Mean Absolute Error: 268059.8015
- Mean S

In [25]:
# Create dictionaries of hyperparameters
knn_params = {"n_neighbors": [2, 3, 10, 20, 40, 50]}

rf_params = {
    "max_depth": [5, 8, 15, None, 10],
    "max_features": [5, 7, 8],
    "min_samples_split": [2, 8, 15, 20],
    "n_estimators": [100, 200, 500, 1000]
}

ada_params={
    "n_estimators":[50, 60, 70, 80],
    "loss":['linear','square','exponential']
}

In [26]:
# Create a list of models with their hyperparameters
randomcv_models = [
    ('KNN', KNeighborsRegressor(), knn_params),
    ("Random Forest", RandomForestRegressor(), rf_params),
    ("Adaboost",AdaBoostRegressor(), ada_params)
]

In [28]:
from sklearn.model_selection import RandomizedSearchCV

# Create a dictionary to store the best parameters
model_param = {}

# Loop through the models and find the best parameters
for name, model, params in randomcv_models:
    random = RandomizedSearchCV(
        estimator=model,
        param_distributions=params,
        n_iter=100,
        cv=3,
        verbose=1,
        n_jobs=-1
    )
    
    # Fit the model
    random.fit(X_train, y_train)
    
    # Store the best parameters
    model_param[name] = random.best_params_

# Print the best parameters
for model_name in model_param:
    print(f"--------------------- Best Params for {model_name} ------------------------")
    print(model_param[model_name])

/Users/ricardo-rojas/Library/Mobile Documents/com~apple~CloudDocs/machine_learning/ml_engineer/venv/lib/python3.12/site-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits


/Users/ricardo-rojas/Library/Mobile Documents/com~apple~CloudDocs/machine_learning/ml_engineer/venv/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/ricardo-rojas/Library/Mobile Documents/com~apple~CloudDocs/machine_learning/ml_engineer/venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
81 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
46 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/ricardo-rojas/Library/Mobile Documents/co

Fitting 3 folds for each of 12 candidates, totalling 36 fits


/Users/ricardo-rojas/Library/Mobile Documents/com~apple~CloudDocs/machine_learning/ml_engineer/venv/lib/python3.12/site-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 12 is smaller than n_iter=100. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


--------------------- Best Params for KNN ------------------------
{'n_neighbors': 10}
--------------------- Best Params for Random Forest ------------------------
{'n_estimators': 100, 'min_samples_split': 2, 'max_features': 8, 'max_depth': None}
--------------------- Best Params for Adaboost ------------------------
{'n_estimators': 60, 'loss': 'linear'}


In [29]:
# Create a dictionary of models with their best parameters
models = {
    "Random Forest Regressor": RandomForestRegressor(
        n_estimators=100, 
        min_samples_split=2, 
        max_features=8, 
        max_depth=None, 
        n_jobs=-1),
     "K-Neighbors Regressor": KNeighborsRegressor(
        n_neighbors=10, 
        n_jobs=-1),
     "Adaboost":AdaBoostRegressor(
        n_estimators=60,
        loss='linear',)
}

# Loop through the models and evaluate the model performance
for i in range(len(list(models))):
    # Get the model
    model = list(models.values())[i]
    
    # Fit the model
    model.fit(X_train, y_train)

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate the model
    model_train_mae, model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae, model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    
    # Print the results
    print(list(models.keys())[i])
    
    print('Model Performance for Training set')
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- Mean Squared Error: {:.4f}".format(model_train_mse))
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- R2 Score: {:.4f}".format(model_train_r2))
    print("- Adjusted R2 Score: {:.4f}".format(1 - (1-model_train_r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)))

    print('-'*55)
    
    print('Model Performance for Test set')
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- Mean Squared Error: {:.4f}".format(model_test_mse))
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    print("- Adjusted R2 Score: {:.4f}".format(1 - (1-model_test_r2)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)))
    
    print('='*55)
    print('\n')

Random Forest Regressor
Model Performance for Training set
- Mean Absolute Error: 39188.9391
- Mean Squared Error: 15234740100.1955
- Root Mean Squared Error: 123429.0894
- R2 Score: 0.9812
- Adjusted R2 Score: 0.9812
-------------------------------------------------------
Model Performance for Test set
- Mean Absolute Error: 99623.3041
- Mean Squared Error: 48293086090.7381
- Root Mean Squared Error: 219756.8795
- R2 Score: 0.9358
- Adjusted R2 Score: 0.9356


K-Neighbors Regressor
Model Performance for Training set
- Mean Absolute Error: 103458.0062
- Mean Squared Error: 132092390124.1077
- Root Mean Squared Error: 363445.1680
- R2 Score: 0.8371
- Adjusted R2 Score: 0.8369
-------------------------------------------------------
Model Performance for Test set
- Mean Absolute Error: 117441.8180
- Mean Squared Error: 69624092684.2767
- Root Mean Squared Error: 263863.7768
- R2 Score: 0.9075
- Adjusted R2 Score: 0.9071


Adaboost
Model Performance for Training set
- Mean Absolute Error: 